## Convert Bioschemas validation rules to be DDE-validator default-compatible

This script converts the property mappings into a DDE-suitable format

Format can be seen here https://github.com/biothings/discovery-app/blob/vue3-app/nuxt-app/store/modules/editor_options/validation_options.js

  {
    _id: "01233bio",
    title: "text",
    color: "#097969",
    validation: {
      type: "string",
    },
    belongs_to: "bioschemas",
  },

In [2]:
import os
import json
import pandas
import pathlib
import yaml
import requests

In [4]:
import random
datapath = os.path.join('results','mappings')
one_prop = os.path.join(datapath,'one_prop_rule.txt')
many_prop = os.path.join(datapath,'many_prop_rule.txt')

one_prop_json = json.load(open(one_prop,'rb'))
many_prop_json = json.load(open(many_prop,'rb'))

prop_keys = list(one_prop_json.keys())

dde_list = []

idlist = set()

for eachkey in prop_keys:
    tmpidlist = []
    while len(tmpidlist)<3:
        tmpid = eachkey[0:2]+random.choice('abcdefgh')+random.choice('pqrstuvwx')+str(random.randint(100,999))+'bio'
        if tmpid not in idlist:
            tmpidlist.append(tmpid)
            idlist.add(tmpid)
    dde_one = {}
    dde_one['_id'] = tmpidlist[0]
    dde_one['title'] = eachkey
    dde_one['color'] = '#097969'
    dde_one['validation'] = one_prop_json[eachkey]
    dde_one['belongs_to'] = 'bioschemas'
    dde_list.append(dde_one)
    dde_many = dde_one.copy()
    dde_many['_id'] = tmpidlist[1]
    dde_many['validation'] = many_prop_json[eachkey]
    dde_list.append(dde_many)

print(dde_list)

[{'_id': 'ideu553bio', 'title': 'identifier', 'color': '#097969', 'validation': {'oneOf': [{'@type': 'PropertyValue', 'type': 'object', 'properties': {}, 'required': []}, {'type': 'string'}, {'type': 'string', 'format': 'uri'}]}, 'belongs_to': 'bioschemas'}, {'_id': 'idfr404bio', 'title': 'identifier', 'color': '#097969', 'validation': {'anyOf': [{'@type': 'PropertyValue', 'type': 'object', 'properties': {}, 'required': []}, {'type': 'array', 'items': {'@type': 'PropertyValue', 'type': 'object', 'properties': {}, 'required': []}}, {'type': 'string'}, {'type': 'array', 'items': {'type': 'string'}}, {'type': 'string', 'format': 'uri'}, {'type': 'array', 'items': {'type': 'string', 'format': 'uri'}}]}, 'belongs_to': 'bioschemas'}, {'_id': 'hadq186bio', 'title': 'hasRepresentation', 'color': '#097969', 'validation': {'@type': 'PropertyValue orText orURL', 'type': 'object', 'properties': {}, 'required': []}, 'belongs_to': 'bioschemas'}, {'_id': 'haaq642bio', 'title': 'hasRepresentation', 'c

#### generate property list based on expected type

In [5]:
exp_type_file = os.path.join('results','frequency_tables','base_types.json')
expected_type = json.load(open(exp_type_file,'rb'))
exprop_names = list(expected_type.keys())

## Clean up bad entries
for eachprop in exprop_names:
    if ' ' in eachprop:
        exprop_names.remove(eachprop)
        
fixprops = {'text':'Text','url':'URL','number':'Number','integer':'Integer',
            'boolean':'Boolean','date':'Date','datetime':'DateTime'}
exp_type_list = []

for eachkey in exprop_names:
    tmpdict = expected_type[eachkey]
    exp_one = {}
    tmpidlist=[]
    while len(tmpidlist)<3:
        tmpid = 'exty'+random.choice('vwxyzabcdef')+random.choice('ijklmnopq')+str(random.randint(100,999))+'bio'
        if tmpid not in idlist:
            idlist.add(tmpid)
            tmpidlist.append(tmpid)
    exp_one['_id'] = tmpidlist[0]
    if '@type' in tmpdict.keys():
        exp_one['title'] = tmpdict['@type']
    elif eachkey in fixprops.keys():
        exp_one['title'] = fixprops[eachkey]
    else:
        exp_one['title'] = eachkey
    exp_one['color'] = '#097969'
    exp_one['validation'] = expected_type[eachkey]
    exp_one['belongs_to'] = 'bioschemas'
    exp_type_list.append(exp_one)
    exp_many = exp_one.copy()
    exp_many['_id'] = tmpidlist[1]
    exp_many['validation']  = {"oneOf": [
          tmpdict,
          {
            "type": "array",
            "items": tmpdict
          }
        ]}
    exp_type_list.append(exp_many)

In [6]:
print(exp_type_list)

[{'_id': 'extyaq553bio', 'title': 'Text', 'color': '#097969', 'validation': {'type': 'string'}, 'belongs_to': 'bioschemas'}, {'_id': 'extybq929bio', 'title': 'Text', 'color': '#097969', 'validation': {'oneOf': [{'type': 'string'}, {'type': 'array', 'items': {'type': 'string'}}]}, 'belongs_to': 'bioschemas'}, {'_id': 'extyxl513bio', 'title': 'URL', 'color': '#097969', 'validation': {'type': 'string', 'format': 'uri'}, 'belongs_to': 'bioschemas'}, {'_id': 'extydi371bio', 'title': 'URL', 'color': '#097969', 'validation': {'oneOf': [{'type': 'string', 'format': 'uri'}, {'type': 'array', 'items': {'type': 'string', 'format': 'uri'}}]}, 'belongs_to': 'bioschemas'}, {'_id': 'extyyo198bio', 'title': 'PropertyValue', 'color': '#097969', 'validation': {'@type': 'PropertyValue', 'type': 'object', 'properties': {}, 'required': []}, 'belongs_to': 'bioschemas'}, {'_id': 'extyxj593bio', 'title': 'PropertyValue', 'color': '#097969', 'validation': {'oneOf': [{'@type': 'PropertyValue', 'type': 'object',

#### export the resulting files

In [7]:
export_path = os.path.join(datapath,'for_dde_defaults')
with open(os.path.join(export_path,'by_property.txt'),'w') as f:
    f.write(json.dumps(dde_list, indent=2))

with open(os.path.join(export_path,'by_expectedType.txt'),'w') as f2:
    f2.write(json.dumps(exp_type_list, indent=2))